In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))
import h5py
import matplotlib.pyplot as plt
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import loadmat
from skimage import color
from skimage import io
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from keras.models import Sequential, Model
from keras.optimizers import SGD, RMSprop, Adam, Nadam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.constraints import max_norm
from keras.layers import Dense, Dropout, LSTM
from keras.layers import Activation, Flatten, Input, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D 
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from skimage.feature import hog
from sklearn.preprocessing import OneHotEncoder
from sklearn.multiclass import OneVsRestClassifier
from sklearn.decomposition import PCA
import cv2
# Any results you write to the current directory are saved as output.

In [ ]:
# labels_file = '../input/train_digitStruct.mat'
# f = h5py.File(labels_file)
# ref = f.get('/digitStruct/name').value[32402][0]  # <HDF5 object reference>
# print(f[ref].value)
# plt.imshow(f[ref].value)
# ref = f.get('/digitStruct/name')
# ref.shape


In [ ]:
def load_data(path):
    """ Helper function for loading a MAT-File"""
    data = loadmat(path)
    return data['X'], data['y']

path = '../input/svhntrain/train_32x32.mat'
pathTest = '../input/svhntest/test_32x32.mat'
X_train, y_train = load_data(path)
X_test, y_test = load_data(pathTest)


In [ ]:
# Transposing the the train and test data
# by converting it from  
# (width, height, channels, size) -> (size, width, height, channels)

X_train, y_train = X_train.transpose((3,0,1,2)), y_train[:,0]
X_test, y_test = X_test.transpose((3,0,1,2)), y_test[:,0]

In [ ]:
print("Training Set", X_train.shape)
print('')

print("Test Set", X_test.shape)
print('')

# Calculate the total number of images
num_images = X_train.shape[0]
num_images2 = X_test.shape[0]

print("Total Number of Train Images", num_images)
print("Total Number of Test Images", num_images2)

In [ ]:
#plt.imshow(X_train[3,:,:,])

def plot_images(img, labels, nrows, ncols):
    """ Plot nrows x ncols images
    """
    fig, axes = plt.subplots(nrows, ncols)
    for i, ax in enumerate(axes.flat): 
        if img[i].shape == (32, 32, 3):
            ax.imshow(img[i])
        else:
            ax.imshow(img[i,:,:,0])
        ax.set_xticks([]); ax.set_yticks([])
        #ax.set_title(labels[i])

In [ ]:
plot_images(X_train, y_train, 1, 5)

In [ ]:
y_train[y_train == 10] = 0
y_test[y_test == 10] = 0
print(np.unique(y_train))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True)

fig.suptitle('Class Distribution', fontsize=14, fontweight='bold', y=1.05)

ax1.hist(y_train, bins=10)
ax1.set_title("Training set")
ax1.set_xlim(1, 10)

ax2.hist(y_test, color='g', bins=10)
ax2.set_title("Test set")

fig.tight_layout()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.13, random_state=7)

In [ ]:
train_mean = np.mean(X_train, axis=0) #Shape (32,3)
train_std = np.std(X_train, axis=0) #Shape (32,3)
X_train_norm = (X_train - train_mean) / train_std
print(X_train.shape)
print(train_mean.shape)
print(train_std.shape)

In [ ]:
X_train_norm_array = np.array(X_train_norm)
# y_train_array = np.array(y_train).reshape(len(y_train),1)
y_train_array = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
X_val_array = np.array(X_val)
y_val_array = np.array(y_val)
print(X_train_norm_array.shape)
print(y_train_array.shape)


In [ ]:
print(y_train_array.shape)
x_train_reshaped = X_train_norm_array[:,:,:,0]
X_val_array_new = X_val_array[:,:,:,0]
print(x_train_reshaped.shape)
# x_train_wtf = x_train_reshaped.reshape(x_train_reshaped.shape[0],-1)
# X_val_res = X_val_array_new.reshape(X_val_array_new.shape[0],-1)
x_train_wtf = x_train_reshaped.astype(np.float32).reshape(-1, 32*32)
X_val_res = X_val_array_new.astype(np.float32).reshape(-1, 32*32)
print(x_train_wtf.shape)
print(X_val_res.shape)

In [ ]:
print(y_val_array.shape)

In [ ]:
# # kmeans = KMeans(n_clusters=10, random_state=0).fit(x_train_wtf, y_train_array)
# # svm_result = kmeans.predict(X_val_res)
# # svm_result = np.array(svm_result).reshape(len(svm_result), -1)
# # print(classification_report(y_val_array.reshape(len(y_val_array), 1), svm_result))




################################ SPARSE AUTO ENCODER ####################################
encoding_dim = 3

input_img = Input(shape=(32,32,3))
# add a Dense layer with a L1 activity regularizer
encoded = Dense(encoding_dim, activation='relu',
                activity_regularizer=regularizers.l1(10e-5))(input_img)
decoded = Dense(3, activation='sigmoid')(encoded)

autoencoder = Model(input_img, decoded)

encoder = Model(input_img, encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

autoencoder.fit(X_train_norm, X_train_norm,
                epochs=20,
                batch_size=256,
                shuffle=True,
                validation_data=(X_val_array, X_val_array))

encoded_imgs = encoder.predict(X_val_array)

# n = 10  # how many digits we will display
# plt.figure(figsize=(20, 4))
# for i in range(n):
#     # display original
#     ax = plt.subplot(2, n, i + 1)
#     plt.imshow(X_val_res[i].reshape(32, 32))
#     plt.gray()
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)

#     # display reconstruction
#     ax = plt.subplot(2, n, i + 1 + n)
#     plt.imshow(decoded_imgs[i].reshape(32, 32))
#     plt.gray()
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)
# plt.show()

In [ ]:
# kmeans = KMeans(n_clusters=10, random_state=0).fit(x_train_wtf)
# svm_result = kmeans.predict(X_val_res)
# svm_result = np.array(svm_result).reshape(len(svm_result), -1)
# print(classification_report(y_val_array.reshape(len(y_val_array), 1), svm_result))

# ran_clf =  RandomForestClassifier(n_estimators = 200, max_depth=12)
# ran_clf.fit(x_train_wtf, y_train_array)
# ran_predicted = ran_clf.predict(X_val_res)
# print('Random Forest test accuracy : {}'.format(accuracy_score(y_val_array, ran_predicted)))
# print('=============================================================================')

In [ ]:
encoded_imgs = encoder.predict(X_train_norm)
encoded_imgs.shape

In [ ]:
encoded_imgs_val = encoder.predict(X_val_array)

In [ ]:
x_train_reshaped_encoded = encoded_imgs[:,:,:,0]
x_train_wtf_enc = x_train_reshaped_encoded.astype(np.float32).reshape(-1, 32*32)

In [ ]:
x_val_reshaped_encoded = encoded_imgs_val[:,:,:,0]
x_val_wtf_enc = x_val_reshaped_encoded.astype(np.float32).reshape(-1, 32*32)

In [ ]:
y_train_cnn = to_categorical(y_train_array, 10)
y_val_cnn = to_categorical(y_val_array, 10)
y_test_cnn = to_categorical(y_test, 10)
print(y_train_cnn.shape)
print(y_val_cnn.shape)
print(y_test_cnn.shape)

In [ ]:
from skimage import color
X_train_255 = np.divide(X_train, 255)
X_train_255_arr = np.array(X_train_255)
data_gray = [color.rgb2gray(i) for i in X_train_255_arr]

In [ ]:
plt.imshow(data_gray[10], cmap='gray')

In [ ]:
ppc = 16
hog_images = []
hog_features = []
for image in data_gray:
    fd,hog_image = hog(image, orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(2, 2),block_norm= 'L2',visualise=True)
    hog_images.append(hog_image)
    hog_features.append(fd)

In [ ]:
data_gray_val = [color.rgb2gray(i) for i in X_val_array]

In [ ]:
data_gray_test = [color.rgb2gray(i) for i in np.array(X_test)]

In [ ]:
hog_images_val = []
hog_features_val = []
for image_val in data_gray_val:
    fd_val,hog_image_val = hog(image_val, orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(2, 2),block_norm= 'L2',visualise=True)
    hog_images_val.append(hog_image_val)
    hog_features_val.append(fd_val)

In [ ]:
hog_images_test = []
hog_features_test = []
for image_test in data_gray_test:
    fd_test,hog_image_test = hog(image_test, orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(2, 2),block_norm= 'L2',visualise=True)
    hog_images_test.append(hog_image_test)
    hog_features_test.append(fd_test)

In [ ]:
X_train_255.shape

In [ ]:
hog_features = np.array(hog_features)
hog_features_test_255 = np.divide(hog_features_test, 255)

In [ ]:
hog_features_val = np.array(hog_features_val)

In [ ]:
svc_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="poly", degree=7, coef0=1, C=0.5))
    ])
svc_clf.fit(hog_features, y_train_array)
# svc_clf.fit(x_train_wtf_enc, y_train_array)
svc_clf_predicted = svc_clf.predict(hog_features_test)
print('SVM test accuracy : {}'.format(accuracy_score(np.array(y_test), svc_clf_predicted)))
print('=============================================================================')

In [ ]:
ran_clf =  RandomForestClassifier(n_estimators = 200)
ran_clf.fit(hog_features, y_train_array)
ran_predicted = ran_clf.predict(hog_features_test)
print('Random Forest test accuracy : {}'.format(accuracy_score(y_test, ran_predicted)))
print('=============================================================================')

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb_clf = GaussianNB()
gnb_clf.fit(hog_features, y_train_array)
ran_predicted = gnb_clf.predict(hog_features_val)
print('NB Accuracy test accuracy : {}'.format(accuracy_score(y_val_array, ran_predicted)))
print('=============================================================================')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=40) # default value for n_neighbors is 5
knn_clf.fit(hog_features, y_train_array)
ran_predicted = knn_clf.predict(hog_features_test)
print('NB Accuracy test accuracy : {}'.format(accuracy_score(np.array(y_test), ran_predicted)))
print('=============================================================================')

In [ ]:
kmeans = KMeans(n_clusters=10).fit(hog_features)
cluster_centers = kmeans.cluster_centers_
cluster_labels = kmeans.labels_
svm_result = kmeans.predict(hog_features_val)
print(classification_report(y_val_array, svm_result))

In [ ]:
################################ SPARSE AUTO ENCODER ####################################
encoding_dim = 32

input_img = Input(shape=(1024,))
# add a Dense layer with a L1 activity regularizer
encoded = Dense(encoding_dim, activation='relu',
                activity_regularizer=regularizers.l1(10e-5))(input_img)
decoded = Dense(1024, activation='sigmoid')(encoded)

autoencoder = Model(input_img, decoded)

encoder = Model(input_img, encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

autoencoder.fit(x_train_wtf, x_train_wtf,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(X_val_res, X_val_res))

encoded_imgs = encoder.predict(x_train_wtf)
decoded_imgs = decoder.predict(encoded_imgs)

In [ ]:
len(decoded_imgs)

In [ ]:
#Import Library
from sklearn.ensemble import GradientBoostingClassifier
#Assumed you have, X (predictor) and Y (target) for training data set and x_test(predictor) of test_dataset
# Create Gradient Boosting Classifier object
model= GradientBoostingClassifier(n_estimators=100, learning_rate=1, max_depth=1, random_state=0)
# Train the model using the training sets and check score
model.fit(hog_features, y_train_array)
#Predict Output
predicted= model.predict(hog_features_val)
print('NB Accuracy test accuracy : {}'.format(accuracy_score(y_val_array, predicted)))
print('=============================================================================')

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier()

model.fit(hog_features, y_train_array)

#Make predictions for test data
y_pred = model.predict(hog_features_test)
print('NB Accuracy test accuracy : {}'.format(accuracy_score(np.array(y_test), y_pred)))
print('=============================================================================')